# Transformação de Dados

Anteriormente, vimos como podemos combinar dados de diferentes fontes em um DataFrame unificado. Em geral, após um processo de Integração de dados, colunas com diferentes tipos de dados podem ser geradas. Na Transformação de dados, o principal objetivo é exatamente transformar os dados de diferentes formatos em um formato suportado por modelos de Aprendizado de Máquina. A Transformação de dados possui algumas etapas em comum com a Limpeza de dados, incluindo técnicas de remoção de ruído e discretização de dados. Assim, a seguir, apresentamos as demais etapas envolvidas nesta fase de Pré-processamento.

In [1]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
pd.set_option('display.max_columns', 6)

## Dados não padronizados

Ter dados coletados de diferentes fontes pode gerar um conjunto de dados heterogêneo e não padronizado, contendo atributos de diferentes escalas. O reescalonamento de dados não é uma etapa obrigatória, mas definitivamente é uma boa prática. Muitos métodos de Aprendizado de Máquina são mais eficazes se os atributos dos dados seguem um padrão. Portanto, para garantir que o desempenho de nossos modelos não varie por causa de diferentes magnitudes de recursos, precisamos realizar o reescalonamento de dados. Essencialmente, existem duas técnicas de reescalonamento de dados, que transformam as features para que fiquem em uma mesma escala, magnitude ou intervalo. A seguir, descrevemos e exemplificamos cada uma delas utilizando o conjunto de atributos numéricos que descrem as músicas do Spotify.

In [2]:
# Lendo os dados
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8')

# Selecionando apenas variáveis numéricas
df = data.select_dtypes(include=['number'])
print(df.columns.values) # imprime as colunas restantes

['popularity' 'track_number' 'num_artists' 'num_available_markets'
 'duration_ms' 'key' 'mode' 'time_signature' 'acousticness' 'danceability'
 'energy' 'instrumentalness' 'liveness' 'loudness' 'speechiness' 'valence'
 'tempo']


### Normalização de dados 

Reescalona os atributos numéricos em um intervalo de 0 a 1. Em resumo, o objetivo da normalização é alterar os valores das colunas numéricas no conjunto de dados para uma escala comum, sem distorcer as diferenças nos intervalos de valores. O exemplo a seguir demonstra a normalização de dados, redimensionando cada atributo do *DataFrame* para um intervalo de $[0,1]$. Para isso, subtraímos o valor mínimo de cada coluna e dividimos pelo intervalo, aplicando e escala min-max do *Pandas* usando os métodos `min()` e `max()`.

In [3]:
df_norm = df.copy()  # cópia do DataFrame
for coluna in df_norm.columns:  # Para cada coluna,
    df_norm[coluna] = (  # Normalização min-max
        df_norm[coluna] - df_norm[coluna].min()
    ) / (df_norm[coluna].max() - df_norm[coluna].min())
df_norm.head(3)

,popularity,track_number,num_artists,...,speechiness,valence,tempo
0,0.793814,0.310345,0.0,...,0.015335,0.212314,0.506887
1,0.680412,0.448276,0.0,...,0.176348,0.558386,0.606828
2,0.134021,0.000000,0.0,...,0.243727,0.143312,0.393380


### Padronização de Dados

Reescalona a distribuição de cada atributo para apresentar média igual a zero e um desvio padrão igual a um. Em resumo, cada valor padronizado é calculado subtraindo a média do recurso correspondente e dividindo pelo desvio padrão. O exemplo a seguir demonstra a padronização de dados, transformando os atributos do *DataFrame* em distribuições com uma média igual a 0 e um desvio padrão igual a 1. Para isso, subtraímos a média de cada coluna e dividimos pelo desvio padrão, usando os métodos `mean()` e `std()` do *Pandas*.

In [4]:
df_padron = df.copy()  # cópia do DataFrame
for coluna in df_padron.columns:  # Para cada coluna,
    df_padron[coluna] = (  # Padronização
        df_padron[coluna] - df_padron[coluna].mean()
    ) / (df_padron[coluna].std())
df_padron.head(3)

,popularity,track_number,num_artists,...,speechiness,valence,tempo
0,0.448125,1.007112,-0.644346,...,-0.802165,-1.217354,0.122480
1,-0.164943,1.806576,-0.644346,...,0.462704,0.221431,0.658761
2,-3.118819,-0.791681,-0.644346,...,0.992015,-1.504228,-0.486598


## Feature Engineering (Engenharia de features)

Consiste em criar ou remover features em um conjunto de dados procurando melhorar o desempenho dos modelos de Aprendizado de Máquina. Muitas das técnicas de Transformação e Limpeza de dados podem ser consideradas técnicas de Feature Engineering, incluindo métodos de reescalonamento, discretização e redução de dimensionalidade. Portanto, a seguir, cobrimos duas técnicas ainda não abordadas neste capítulo. Primeiro, criamos atributos a partir de dados categóricos e, em seguida, a partir de datas.

In [5]:
# Selecionando apenas variáveis não-numéricas
df = data[['explicit', 'song_type']].astype('category')
df.head()

,explicit,song_type
0,False,Solo
1,True,Solo
2,False,Solo
3,False,Solo
4,True,Collaboration


In [6]:
# Retornando os dtypes do DataFrame
df.dtypes

explicit     category
song_type    category
dtype: object

### Dados categóricos

São variáveis que só podem assumir um número limitado e, geralmente, fixo de valores possíveis. Por exemplo, em nosso conjunto de músicas do Spotify, encontramos dados categóricos sobre o tipo e a expliciticidade das músicas. Ou seja, cada unidade de observação (i.e., música) é atribuída a um determinado grupo ou categoria nominal. Apesar de estarem frequentemente presentes em conjuntos de dados reais, a maioria dos algoritmos de Aprendizado de Máquina tem dificuldade para lidar com dados categóricos. Portanto, é preciso convertê-los em formas numéricas antes de serem usados. Uma das abordagens mais simples e comuns é converter variáveis categóricas em *dummies* ou indicadores. Para isso, podemos utilizar a função `get_dummies()` do *Pandas*, conforme o exemplo a seguir.

In [7]:
# Convertendo em dummies
df_dummies = pd.get_dummies(df)
df_dummies.head()

,explicit_False,explicit_True,song_type_Collaboration,song_type_Solo
0,1,0,0,1
1,0,1,0,1
2,1,0,0,1
3,1,0,0,1
4,0,1,1,0


In [8]:
df_dummies.dtypes

explicit_False             uint8
explicit_True              uint8
song_type_Collaboration    uint8
song_type_Solo             uint8
dtype: object

### Variáveis de data

São um tipo especial de dados categóricos. Embora, aparentemente, uma data nos forneça nada mais do que um momento específico no calendário, quando pré-processada corretamente, ela pode enriquecer muito o conjunto de dados. Nos exemplos a seguir, convertemos datas em formatos amigáveis, extraindo recursos e criando novas variáveis que podem ser usadas na análise de um modelo. Especificamente, extraímos recursos da data de lançamento das músicas do Spotify, utilizando métodos do *Pandas*. 

In [9]:
cols = ['song_id', 'song_name', 'release_date']
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8', usecols=cols)

In [10]:
# Convertendo a data de lançamento em 'datetime'
data['release_date'] = pd.to_datetime(
    data['release_date'])
data.dtypes

song_id                 object
song_name               object
release_date    datetime64[ns]
dtype: object

In [11]:
data['day'] = data['release_date'].dt.day  # dia
data['month'] = data['release_date'].dt.month  # mês
data['year'] = data['release_date'].dt.year  # ano
data.head(3)

,song_id,song_name,release_date,day,month,year
0,2rRJrJEo19S2J82BDsQ3F7,Falling,2020-03-26,26,3,2020
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,2020-03-06,6,3,2020
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,2020-07-30,30,7,2020


## Dados desbalanceados

São uma ocorrência muito comum em domínios do mundo real, especialmente em modelos de classificação. Pode-se dizer que dados estão desbalanceados quando existe uma proporção desproporcional de observações de cada classe. Os impactos de dados desbalanceados não geram um erro imediato ao construir e executar seu modelo. Porém, os resultados podem ser ilusórios, dado que a maioria das técnicas de Aprendizado de Máquina funcionam melhor quando o número de amostras em cada classe é equilibrado. No exemplo a seguir, nós usamos as features acústicas das músicas do Spotify para prever se uma música é uma colaboração ou não. Especificamente, modelamos o problema como uma classificação binária, cada música é setada como 1 se a for uma colaboração, ou 0 caso contrário.

In [12]:
cols = [
    'duration_ms', 'key', 'mode', 'time_signature', 'acousticness',
    'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness',
    'speechiness', 'valence', 'tempo', 'song_type'
]

# Selecionando algumas colunas do conjunto de músicas
data = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv', encoding='utf-8', usecols=cols)

In [13]:
data.head(3)

,song_type,duration_ms,key,...,speechiness,valence,tempo
0,Solo,159381,10,...,0.0364,0.236,127.087
1,Solo,188800,9,...,0.1750,0.562,142.933
2,Solo,180950,10,...,0.2330,0.171,109.090


In [14]:
data['song_type'] = [ # Mapeia 'song_type' em 0 e 1
    int(x == 'Collaboration') for x in data.song_type] 
data['song_type'].value_counts() # qtd de cada classe

0    752
1    532
Name: song_type, dtype: int64

A função `value_counts()` computa os valores únicos de cada classe. Como resultado, apenas cerca de 41,4% das observações (i.e., 532 músicas) são colaborações. Portanto, se desejamos prever sempre a classe 0, músicas solo, obteríamos uma precisão de 58,6%. A seguir, nós testamos esse cenário criando um modelo linear de regressão logística, usando o módulo `sklearn.linear_model` e a classe `LogisticRegression`.

In [15]:

y = data.song_type # treino (X)
X = data.drop('song_type', axis=1) # teste (y)
logr = LogisticRegression().fit(X, y) # treinando o modelo de regressão logística
pred_y_0 = logr.predict(X)
print(accuracy_score(pred_y_0, y)) # acurácia do modelo
print(np.unique(pred_y_0)) # classes previstas pelo modelo

0.5856697819314641
[0]


O resultado anterior indica que nosso modelo apresentou uma precisão geral de 58,6%. Ou seja, como consequência dos dados desbalanceados, nosso modelo prevê apenas uma classe. Isso significa que o modelo ignora completamente a classe minoritária (i.e., colaborações) em favor da classe majoritária (i.e., músicas solo). A seguir, aplicamos a técnica de *Downsampling*, que remove aleatoriamente observações da classe majoritária para evitar que seu sinal domine o algoritmo de aprendizagem. Primeiro, separamos as observações de cada classe em diferentes *DataFrames*. Em seguida, reamostramos a classe majoritária sem substituição, definindo o número de amostras para corresponder ao da classe minoritária. Finalmente, concatenamos o *DataFrame* da classe minoritária com o *DataFrame* resultante.

In [16]:

df_majority = data[data.song_type == 0] # classe majoritária
df_minority = data[data.song_type == 1] # classe minoritária
df_majority_downsampled = resample(
    df_majority, replace=False,  # amostra sem substituição
    n_samples=532,  # para corresponder à classe minoritária
    random_state=123)  # garantindo reprodutibilidade
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled.song_type.value_counts()

1    532
0    532
Name: song_type, dtype: int64

Desta vez, apesar do novo *DataFrame* ter menos observações do que o original, a proporção das duas classes agora está balanceada. Podemos, então, treinar novamente nosso modelo de regressão logística. Conforme o exemplo a seguir, após o balanceamento dos dados, o modelo deixa de prever apenas uma classe. Embora a precisão tenha diminuído, a métrica de avaliação se torna mais significativa.

In [17]:
y = df_downsampled.song_type # treino (X)
X = df_downsampled.drop('song_type', axis=1) # teste (y)
logr = LogisticRegression().fit(X, y) # Treinando o modelo de regressão logística
pred_y_1 = logr.predict(X)
print(np.unique(pred_y_1)) # acurácia do modelo
print(accuracy_score(y, pred_y_1)) #classes previstas pelo modelo

[0 1]
0.49154135338345867


## Conclusão

Este notebook apresentou como fazer a transformação de dados de diferentes formatos.

O próximo notebook ([3.4.Reducao.ipynb](3.4.Reducao.ipynb)) apresenta como aplicar técnicas de redução de dados para auxiliar a análise de dados com alta dimensionalidade.